In [1]:
import os
import json
import string
import numpy as np 
import pandas as pd
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
# from lightgbm import LGBMClassifier
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

In [2]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported

# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
train = pl.read_csv("train.csv")
test = pl.read_csv("test.csv")

In [4]:
train = train.with_columns(pl.col("category").str.to_lowercase(), 
                    pl.col("sub_category").str.to_lowercase().fill_null("NULL"), 
                    pl.col("crimeaditionalinfo").str.to_lowercase(),
                    (pl.struct(["category", "sub_category"])
                    .map_elements(lambda e: json.dumps({"category": e["category"], "sub_category": e["sub_category"]})))
                    .alias("output"))
test = test.with_columns(pl.col("category").str.to_lowercase(), 
                    pl.col("sub_category").str.to_lowercase().fill_null("NULL"), 
                    pl.col("crimeaditionalinfo").str.to_lowercase(),
                    (pl.struct(["category", "sub_category"])
                    .map_elements(lambda e: json.dumps({"category": e["category"], "sub_category": e["sub_category"]})))
                    .alias("output"))

### For Null text use category = "online financial fraud",  sub_category = "upi related frauds"

In [5]:
# train.filter(pl.col("len").is_null()).group_by(["category", "sub_category"]).len()

In [6]:
train = train.filter(pl.col("crimeaditionalinfo").is_not_null())
test = test.filter(pl.col("crimeaditionalinfo").is_not_null())

In [7]:
max_seq_length = 4000
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 32,
    loftq_config = None,
)
print(model.print_trainable_parameters())


==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce GTX 1650 Ti. Max memory: 4.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`
Unsloth 2024.10.7 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


trainable params: 11,272,192 || all params: 1,509,754,880 || trainable%: 0.7466
None


In [10]:
import json
data_prompt = """Analyze the provided text from a Cybercrime Prevention Assisatant perspective. Identify the category and sub_category of the complaint reported.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompt(df):
    inputs  = df["crimeaditionalinfo"]
    outputs = df["output"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


In [11]:
training_data = Dataset.from_pandas(train.to_pandas())
training_data = training_data.map(formatting_prompt, batched=True)


Map:   0%|          | 0/93665 [00:00<?, ? examples/s]

In [ ]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=training_data,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=8,
        num_train_epochs=10,
        fp16=False,
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
    ),
)

trainer.train()


Generating train split: 0 examples [00:00, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,409 | Num Epochs = 40
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 8
\        /    Total batch size = 128 | Total steps = 1,040
 "-____-"     Number of trainable parameters = 11,272,192


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


RuntimeError: CUDA driver error: out of memory

In [ ]:
model = FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    data_prompt.format(
        #instructions
        text,
        #answer
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 5020, use_cache = True)
answer=tokenizer.batch_decode(outputs)
answer = answer[0].split("### Response:")[-1]
print("Answer of the question is:", answer)


In [ ]:
model.save_pretrained("model/1B_finetuned_llama3.2")
tokenizer.save_pretrained("model/1B_finetuned_llama3.2")
